You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a4f4a8d9-45ba-4918-b9fd-f377a225981b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Title: Unveiling the Power of Artificial Intelligence: A Deep Dive into the Latest Trends and Key Players      │
│                                                                                                                 │
│  Outline:                                                                                                       │
│  I. Introduction                                                                                                │
│  - Definition of Artificial Intelligence (AI)                                                                   │
│  - Importance of AI in various industries                                                                       │
│  - Brief overview of the article's key points                                                                   │
│                                                                                                                 │
│  II. Latest Trends in Artificial Intelligence                                                                   │
│  A. Machine Learning and Deep Learning advancements                                                             │
│  B. Natural Language Processing (NLP) developments                                                              │
│  C. AI-powered automation in businesses                                                                         │
│  D. Ethical considerations in AI applications                                                                   │
│                                                                                                                 │
│  III. Key Players in the AI Industry                                                                            │
│  A. Tech Giants                                                                                                 │
│  - Google                                                                                                       │
│  - Amazon                                                                                                       │
│  - Microsoft                                                                                                    │
│  B. AI Startups                                                                                                 │
│  - OpenAI                                                                                                       │
│  - UiPath                                                                                                       │
│  C. Academic & Research Institutions                                                                            │
│  - DeepMind                                                                                                     │
│  - MIT AI Lab                                                                                                   │
│  D. Government Initiatives                                                                                      │
│  - DARPA                                                                                                        │
│  - European AI Alliance                                                                                         │
│                                                                                                                 │
│  IV. Noteworthy News in Artificial Intelligence        

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cc33a609-beb7-4e6b-bb04-562f53000e5c                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Would you like to view your execution traces? [y/N] (20s timeout): 



╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a4f4a8d9-45ba-4918-b9fd-f377a225981b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Unveiling the Power of Artificial Intelligence: A Deep Dive into the Latest Trends and Key     │
│  Players                                                                                                        │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  Artificial Intelligence (AI) has significantly transformed various industries by empowering machines to        │
│  execute tasks that traditionally demanded human intelligence. Ranging from healthcare to finance, AI stands    │
│  as a pivotal driver of innovation. This article meticulously explores the most recent trends and principal     │
│  figures propelling the AI domain, delivering valuable insights into the advancements and implications of this  │
│  groundbreaking technology.                                                                                     │
│                                                                                                                 │
│  ## Latest Trends in Artificial Intelligence                                                                    │
│                                                                                                                 │
│  ### Machine Learning and Deep Learning Advancements                                                            │
│  The progress in Machine Learning and Deep Learning has revolutionized AI capabilities. These technologies,     │
│  with their capacity to learn from data, have propelled breakthroughs in areas like image and speech            │
│  recognition, autonomous systems, and predictive analytics.                                                     │
│                                                                                                                 │
│  ### Natural Language Processing (NLP) Developments                                                             │
│  Advancements in NLP have significantly augmented machines' ability to comprehend and generate human language.  │
│  NLP innovations have fueled the growth of applications such as chatbots, language translation, and sentiment   │
│  analysis.                                                                                                      │
│                                                                                                                 │
│  ### AI-Powered Automation in Businesses                                                                        │
│  AI-driven automation is reshaping industries by refining processes, amplifying productivity, and curbing       │
│  operational costs. Ranging from robotic process automation to intelligent algorithms, businesses are           │
│  harnessing AI to gain a competitive edge.                                                                      │
│                                                                                                                 │
│  ### Ethical Considerations in AI Applications                                                                  │
│  The ethical dimensions of AI have attracted considera

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [15]:
from IPython.display import Markdown
Markdown(result.raw)

# Unveiling the Power of Artificial Intelligence: A Deep Dive into the Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has significantly transformed various industries by empowering machines to execute tasks that traditionally demanded human intelligence. Ranging from healthcare to finance, AI stands as a pivotal driver of innovation. This article meticulously explores the most recent trends and principal figures propelling the AI domain, delivering valuable insights into the advancements and implications of this groundbreaking technology.

## Latest Trends in Artificial Intelligence

### Machine Learning and Deep Learning Advancements
The progress in Machine Learning and Deep Learning has revolutionized AI capabilities. These technologies, with their capacity to learn from data, have propelled breakthroughs in areas like image and speech recognition, autonomous systems, and predictive analytics.

### Natural Language Processing (NLP) Developments
Advancements in NLP have significantly augmented machines' ability to comprehend and generate human language. NLP innovations have fueled the growth of applications such as chatbots, language translation, and sentiment analysis.

### AI-Powered Automation in Businesses
AI-driven automation is reshaping industries by refining processes, amplifying productivity, and curbing operational costs. Ranging from robotic process automation to intelligent algorithms, businesses are harnessing AI to gain a competitive edge.

### Ethical Considerations in AI Applications
The ethical dimensions of AI have attracted considerable scrutiny. Matters such as algorithmic bias, data privacy, and decision-making transparency have sparked dialogues about the responsible integration of AI technologies.

## Key Players in the AI Industry

### Tech Giants
Industry heavyweights like Google, Amazon, and Microsoft spearhead AI innovation. Their substantial investments in research and development have culminated in groundbreaking AI solutions that permeate diverse sectors.

### AI Startups
Emerging ventures in AI, such as OpenAI and UiPath, are disrupting the industry with specialized expertise and nimble product development. These startups bring fresh perspectives and inventive solutions to the AI landscape.

### Academic & Research Institutions
Academic powerhouses like DeepMind and MIT AI Lab are at the vanguard of cutting-edge AI research. Their collaborations with industry counterparts and governmental entities play a pivotal role in advancing AI technologies.

### Government Initiatives
Government bodies like DARPA and the European AI Alliance are instrumental in shaping AI policies and regulations. By fostering public-private sector partnerships, these initiatives aim to steer AI development towards ethical and advantageous outcomes.

## Noteworthy News in Artificial Intelligence

### Breakthroughs in AI Healthcare Applications
AI is reshaping healthcare through advancements in diagnostics, personalized medicine, and patient care. From analyzing medical images to predicting disease outcomes, AI-powered solutions are enhancing clinical decision-making.

### AI-Driven Innovations in Autonomous Vehicles
The automotive sector is experiencing a profound transformation with AI-infused features in autonomous vehicles. Self-driving cars, advanced driver-assistance systems, and intelligent traffic management are redefining transportation norms through AI technologies.

### Ethical Controversies Surrounding AI Bias and Transparency
The widespread adoption of AI has raised apprehensions regarding algorithmic bias and the absence of transparency in decision-making processes. Striking a balance between innovation and ethical considerations remains a critical challenge in the AI realm.

### Investments and Acquisitions Shaping the AI Landscape
The surge in investments and acquisitions in AI startups and technologies is reshaping the industry landscape. From venture capital funding to strategic alliances, financial activities are propelling the growth and consolidation of AI enterprises.

## Target Audience
- Technology enthusiasts intrigued by AI advancements
- Business professionals seeking to integrate AI into their organizations
- Students and researchers in the Artificial Intelligence domain
- General public intrigued by the societal impact of AI

## SEO Keywords

Artificial Intelligence trends, Top AI companies, AI news updates, Machine Learning breakthroughs, Impact of AI technology

## Resources
1. Forbes - AI section for the latest news and insights
2. MIT Technology Review - Extensive articles on AI advancements
3. Gartner Research - Reports on emerging AI trends and forecasts
4. AI conferences and summits for real-time updates and networking opportunities

**Call to Action:** Stay updated and engaged with the ever-evolving realm of Artificial Intelligence! Subscribe to our newsletter for regular updates on AI trends, key players, and industry insights.

In conclusion, this article presents a comprehensive exploration of Artificial Intelligence, delving into the latest trends, key industry players, significant news, and ethical considerations. By embracing AI advancements and comprehending their impact, individuals and enterprises can navigate the dynamic AI landscape with informed decisions and a forward-looking outlook.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).